# Step 1a: Creating a DataFrame from the category.json file

The following cell is using the SparkSession instance created by Databricks when you started your cluster. Your Jupyter notebook on Databricks follows the common practice of naming the variable for the SparkSession as "spark".  

The SparkSession has a `read` method as we discussed before, and that `read` method returns a `DataFrameReader`, and one of the methods of a `DataFrameReader` is the `json` method which reads a JSON file and returns a `DataFrame`.  See the <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader" target="_blank">PySpark documentation</a> for details on the `DataFrameReader`.  

The  JSON file with the categories was previously downloaded to our accounts in an earlier exercise from 
<a href="https://www.yelp.com/developers/documentation/v3/all_category_list" target="_blank">this</a> page in Yelp's API documentation.

In [0]:
df_categories = spark.read.json("/yelp/categories.json")
print( "number of categories:", df_categories.count() )
df_categories.show()
df_categories.printSchema()

number of categories: 1565
+------------------+--------------------+-----------------+---------------+-------------------+
 alias| country_blacklist|country_whitelist| parents| title|
+------------------+--------------------+-----------------+---------------+-------------------+
 3dprinting| null| null|[localservices]| 3D Printing|
 abruzzese| null| [IT]| [italian]| Abruzzese|
 absinthebars| null| [CZ]| [bars]| Absinthe Bars|
 acaibowls|[AR, TR, CL, PL, ...| null| [food]| Acai Bowls|
 accessories| null| null| [fashion]| Accessories|
 accountants| null| null| [professional]| Accountants|
 acnetreatment| null| null| [beautysvc]| Acne Treatment|
 active| null| null| []| Active Life|
 acupuncture| null| null| [health]| Acupuncture|
 addictionmedicine| [JP]| null| [physicians]| Addiction Medicine|
 adoptionservices| null| null|[localservices]| Adoption Services|
 adult| null| null| [shopping]| Adult|
 adultedu| null| null| [education]| Adult Education|
adultentertainment| null| null| [nightlife]|Adult Entertainment|
 advertising| null| null| [professional]| Advertising|
 aerialfitness| null| null| [fitness]| Aerial Fitness|
 aerialtours| null| null| [tours]| Aerial Tours|
 aestheticians|[AR, CH, CL, BE, ...| null| [medicalspa]| Aestheticians|
 afghani| [MX, TR]| null| [restaurants]| Afghan|
 african| [TR]| null| [restaurants]| African|
+------------------+--------------------+-----------------+---------------+-------------------+
only showing top 20 rows

root
-- alias: string (nullable = true)
-- country_blacklist: array (nullable = true)
 |-- element: string (containsNull = true)
-- country_whitelist: array (nullable = true)
 |-- element: string (containsNull = true)
-- parents: array (nullable = true)
 |-- element: string (containsNull = true)
-- title: string (nullable = true)

# Step 1b: Creating a DataFrame for the Business Data

We have worked with this data in prior exercises, here we are going to start using SQL queries.

In [0]:
df_business = spark.read.json('/yelp/business.bz2')
print ("record count:", df_business.count() )
df_business.printSchema()

record count: 209393
root
-- address: string (nullable = true)
-- attributes: struct (nullable = true)
 |-- AcceptsInsurance: string (nullable = true)
 |-- AgesAllowed: string (nullable = true)
 |-- Alcohol: string (nullable = true)
 |-- Ambience: string (nullable = true)
 |-- BYOB: string (nullable = true)
 |-- BYOBCorkage: string (nullable = true)
 |-- BestNights: string (nullable = true)
 |-- BikeParking: string (nullable = true)
 |-- BusinessAcceptsBitcoin: string (nullable = true)
 |-- BusinessAcceptsCreditCards: string (nullable = true)
 |-- BusinessParking: string (nullable = true)
 |-- ByAppointmentOnly: string (nullable = true)
 |-- Caters: string (nullable = true)
 |-- CoatCheck: string (nullable = true)
 |-- Corkage: string (nullable = true)
 |-- DietaryRestrictions: string (nullable = true)
 |-- DogsAllowed: string (nullable = true)
 |-- DriveThru: string (nullable = true)
 |-- GoodForDancing: string (nullable = true)
 |-- GoodForKids: string (nullable = true)
 |-- GoodForMeal: string (nullable = true)
 |-- HairSpecializesIn: string (nullable = true)
 |-- HappyHour: string (nullable = true)
 |-- HasTV: string (nullable = true)
 |-- Music: string (nullable = true)
 |-- NoiseLevel: string (nullable = true)
 |-- Open24Hours: string (nullable = true)
 |-- OutdoorSeating: string (nullable = true)
 |-- RestaurantsAttire: string (nullable = true)
 |-- RestaurantsCounterService: string (nullable = true)
 |-- RestaurantsDelivery: string (nullable = true)
 |-- RestaurantsGoodForGroups: string (nullable = true)
 |-- RestaurantsPriceRange2: string (nullable = true)
 |-- RestaurantsReservations: string (nullable = true)
 |-- RestaurantsTableService: string (nullable = true)
 |-- RestaurantsTakeOut: string (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- WheelchairAccessible: string (nullable = true)
 |-- WiFi: string (nullable = true)
-- business_id: string (nullable = true)
-- categories: string (nullable = true)
-- city: string (nullable = true)
-- hours: struct (nullable = true)
 |-- Friday: string (nullable = true)
 |-- Monday: string (nullable = true)
 |-- Saturday: string (nullable = true)
 |-- Sunday: string (nullable = true)
 |-- Thursday: string (nullable = true)
 |-- Tuesday: string (nullable = true)
 |-- Wednesday: string (nullable = true)
-- is_open: long (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- name: string (nullable = true)
-- postal_code: string (nullable = true)
-- review_count: long (nullable = true)
-- stars: double (nullable = true)
-- state: string (nullable = true)

# Step 2: Create Temporary Views
  We cannot query a DataFrame directly using the `spark.sql()` methof of our spark session.  Instead we first need to create a temporary view
  of the DataFrame.  Similar to a DataFrame, a temporary view exists in memory, and similar to DataFrames, Spark keeps track of the "recipe" for 
  creating the temporary view if it needs to remove it from memory (if it runs out of space).
  In the following cell, we create temporary views for the `df_categories` and `df_business` DataFrames.
  
  ![Temporary View](http://www.sjsu.edu/people/scott.jensen/AoMBigDataWorkshop/TemporaryView.png)
  
  **Remember:** the DataFrame method for creating a view is long (`createOrReplaceTempView`), but if your DataFrame already exists, after you type the 
  DataFrame's name and the "dot", just type "cr" for the start of the method name and press the tab key.  Spark will display a list of methods
  for the DataFrame that start with "cr", just be careful that you pick the correct method.

In [0]:
df_categories.createOrReplaceTempView("categories")
df_business.createOrReplaceTempView("business")

## What the Category Data Means

Following is a description of the fields in the categories data:
* <p>**alias**:  This is similar to the `title` field, but it's all lower case and there are no spaces in the name.  This is also the value used in the `parents` field to identify the parent-child relationships in the category hierarchy.  For most categories the `alias` and `title` are similar, but check out the category where the `title` is "Fast Food".</p>
* <p>**country_blacklist**:  A category may have a white list or black list, but not both.  This field is either a list of 2-digit country codes or null. if this field contains a list of country codes, then the category can be used in any country except those in the list.  For example, the cuisine "Afghan" has the blacklist \[MX, TR\], which means this category is not used in Mexico or Turkey.<p>
* <p>**country_whitelist**:  If there is a list in this field, the category only is used in those countries.  For example, the category "Absinthe Bars", which is a sub-category under "Bars" has a whitelist with one country \[CZ\] which is the Czech Republic.  This category will only be used to describe businesses in that country.<p>
* <p>**parents**: The categories form a hierarchy, so the broader parent category above each category is a list in this field.  although most categories have one parent category, there are a few with multiple parent categories in the list.  If the list is empty \[ \] for a category (such as "Active Life"), then the category is a top-level category and there is no parent category.<p>
* <p>**title**:  This is the human-readable label for the category and is also the label included in the list of categories for a business in the `categories` field in the Yelp business data.<p>

# Profiling the Category Data

There are a number of questions we could ask about this data:
* What are the top-level categories and how many are there? 
  * We may want to determine the hierarchy for the categories - how deep does it go?
* How many categories have multiple parents?  
* What appears in the business data?
* How many whitelisted categories include only the U.S. ***or*** Canada?
* How many blacklisted categories include only the U.S. ***or*** Canada?

### Step 3a - Structuring: What shows up for categories in the Yelp business data?
We are going to look at PySpark and spark.sql() versions of this query.

In the following cell, we use PySpark's `select` method for the DataFrame to get the following columns for the first 10 records:
* business_id
* name
* state
* attributes
* categories

In [0]:
df_business.select("business_id", "name", "state","attributes","categories").show(10, truncate=60)

+----------------------+----------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
 business_id| name|state| attributes| categories|
+----------------------+----------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
f9NumwFMBDn751xgFiRbNA| The Range At Lake Norman| NC|[,,,,,,, True,, True, {'garage': False, 'street': False, ...| Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping|
Yzvjg0SayhoZgCljUJRF9Q| Carlos Santo, NMD| AZ| [,,,,,,,,,,, True,,,,,,,, True,,,,,,,,,,,,,,,,,,,]|Health & Medical, Fitness & Instruction, Yoga, Active Lif...|
XNoUzKckATkOD1hP6vghZg| Felinus| QC| null| Pets, Pet Services, Pet Groomers|
6OAZjbxqM5ol29BuHsil3w| Nevada House of Hose| NV|[,,,,,,, True,, True, {'garage': False, 'street': False, ...|Hardware Stores, Home Services, Building Supplies, Home &...|
51M2Kk903DFYI6gnB5I6SQ| USE MY GUY SERVICES LLC| AZ| [,,,,,,,,, True,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,]|Home Services, Plumbing, Electricians, Handyman, Contractors|
cKyLV5oWZJ2NudWgqs8VZw| Oasis Auto Center - Gilbert| AZ| [,,,,,,,,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,,,]|Auto Repair, Automotive, Oil Change Stations, Transmissio...|
oiAlXZPIFm2nBCt0DHLu_Q| Green World Cleaners| NV|[,,,,,,, True,, True, {'garage': False, 'street': False, ...| Dry Cleaning & Laundry, Local Services, Laundry Services|
ScYkbYNkDgCneBrD9vqhCQ|Junction Tire & Auto Service| AZ| [,,,,,,,,, True,, False,,,,,,,,,,,,,,,,,,,,,,,,,,,]| Auto Repair, Oil Change Stations, Automotive, Tires|
pQeaRpvuhoEqudo3uymHIQ| The Empanadas House| IL|[,, 'none',,,,, False,, True, None, False,,,,,,,, False,,...|Ethnic Food, Food Trucks, Specialty Food, Imported Food, ...|
EosRKXIGeSWFYWwpkbhNnA| Xtreme Couture| ON|[,,,,,,, False,,, {'garage': False, 'street': False, 'val...| Martial Arts, Gyms, Fitness & Instruction, Active Life|
+----------------------+----------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
only showing top 10 rows

### Step 3b: Query in SQL
We want to select the same 5 columns from the business view.

The query returns a DataFrame, so we can use the same `show` method as we did with PySpark

In [0]:
spark.sql("""
SELECT business_id, name, state, attributes, categories
FROM business
""").show(10, truncate=60)

+----------------------+----------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
 business_id| name|state| attributes| categories|
+----------------------+----------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
f9NumwFMBDn751xgFiRbNA| The Range At Lake Norman| NC|[,,,,,,, True,, True, {'garage': False, 'street': False, ...| Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping|
Yzvjg0SayhoZgCljUJRF9Q| Carlos Santo, NMD| AZ| [,,,,,,,,,,, True,,,,,,,, True,,,,,,,,,,,,,,,,,,,]|Health & Medical, Fitness & Instruction, Yoga, Active Lif...|
XNoUzKckATkOD1hP6vghZg| Felinus| QC| null| Pets, Pet Services, Pet Groomers|
6OAZjbxqM5ol29BuHsil3w| Nevada House of Hose| NV|[,,,,,,, True,, True, {'garage': False, 'street': False, ...|Hardware Stores, Home Services, Building Supplies, Home &...|
51M2Kk903DFYI6gnB5I6SQ| USE MY GUY SERVICES LLC| AZ| [,,,,,,,,, True,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,]|Home Services, Plumbing, Electricians, Handyman, Contractors|
cKyLV5oWZJ2NudWgqs8VZw| Oasis Auto Center - Gilbert| AZ| [,,,,,,,,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,,,]|Auto Repair, Automotive, Oil Change Stations, Transmissio...|
oiAlXZPIFm2nBCt0DHLu_Q| Green World Cleaners| NV|[,,,,,,, True,, True, {'garage': False, 'street': False, ...| Dry Cleaning & Laundry, Local Services, Laundry Services|
ScYkbYNkDgCneBrD9vqhCQ|Junction Tire & Auto Service| AZ| [,,,,,,,,, True,, False,,,,,,,,,,,,,,,,,,,,,,,,,,,]| Auto Repair, Oil Change Stations, Automotive, Tires|
pQeaRpvuhoEqudo3uymHIQ| The Empanadas House| IL|[,, 'none',,,,, False,, True, None, False,,,,,,,, False,,...|Ethnic Food, Food Trucks, Specialty Food, Imported Food, ...|
EosRKXIGeSWFYWwpkbhNnA| Xtreme Couture| ON|[,,,,,,, False,,, {'garage': False, 'street': False, 'val...| Martial Arts, Gyms, Fitness & Instruction, Active Life|
+----------------------+----------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
only showing top 10 rows

# Step 4a: How many top-level categories are there?
Take a look at your earlier code for showing the first 20 rows of the category data.  "Active Life" is one of the top-level categories (and the first category listed on <a href="https://www.yelp.com/developers/documentation/v3/category_list" target="_blank">this</a> page.  What's different about the list in the `parents` column for that category?

In PySpark there's a <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.size" target="_blank">`size` function</a> that will return the number of elements in an array.  The function takes one parameter, the name of the column containing the array.  As when using any PySpark functions, we will need to include an import statement to bring in the functions module.  As usual, we will alias the module's long name as "f".

In the following code cell, since we are writing Python code in our PySpark filter method, we use double equal signs `==` to check for equivalence.  Keep in mind that in Python (and most computing languages), a single `=` is the assignment operator to assign a value to a variable.

In [0]:
import pyspark.sql.functions as f

df_categories.filter( f.size(df_categories.parents) == 0 ).show(30, truncate=False)

+------------------+-----------------+-----------------+-------+----------------------------+
alias |country_blacklist|country_whitelist|parents|title |
+------------------+-----------------+-----------------+-------+----------------------------+
active |null |null |[] |Active Life |
arts |null |null |[] |Arts & Entertainment |
auto |null |null |[] |Automotive |
beautysvc |null |null |[] |Beauty & Spas |
bicycles |null |[CZ, PL, DK, PT] |[] |Bicycles |
education |null |null |[] |Education |
eventservices |null |null |[] |Event Planning & Services |
financialservices |null |null |[] |Financial Services |
food |null |null |[] |Food |
health |null |null |[] |Health & Medical |
homeservices |null |null |[] |Home Services |
hotelstravel |null |null |[] |Hotels & Travel |
localflavor |null |null |[] |Local Flavor |
localservices |null |null |[] |Local Services |
massmedia |null |null |[] |Mass Media |
nightlife |null |null |[] |Nightlife |
pets |null |null |[] |Pets |
professional |null |null |[] |Professional Services |
publicservicesgovt|null |null |[] |Public Services & Government|
religiousorgs |null |null |[] |Religious Organizations |
restaurants |null |null |[] |Restaurants |
shopping |null |null |[] |Shopping |
+------------------+-----------------+-----------------+-------+----------------------------+

### Step 4b: Selecting the top-level categories using SQL

In the following cell, we show the same query using the `spark.sql()` method.

As a convention, capitalize all of the clauses (e.g., SELECT, FROM, WHERE).  Although it will run lower-case, this will make your
queries easier to read.  The same goes for using functions within your SQL queries.

In [0]:
spark.sql("""
SELECT *
FROM categories
WHERE SIZE(parents) = 0
""").show(30, truncate=False)

+------------------+-----------------+-----------------+-------+----------------------------+
alias |country_blacklist|country_whitelist|parents|title |
+------------------+-----------------+-----------------+-------+----------------------------+
active |null |null |[] |Active Life |
arts |null |null |[] |Arts & Entertainment |
auto |null |null |[] |Automotive |
beautysvc |null |null |[] |Beauty & Spas |
bicycles |null |[CZ, PL, DK, PT] |[] |Bicycles |
education |null |null |[] |Education |
eventservices |null |null |[] |Event Planning & Services |
financialservices |null |null |[] |Financial Services |
food |null |null |[] |Food |
health |null |null |[] |Health & Medical |
homeservices |null |null |[] |Home Services |
hotelstravel |null |null |[] |Hotels & Travel |
localflavor |null |null |[] |Local Flavor |
localservices |null |null |[] |Local Services |
massmedia |null |null |[] |Mass Media |
nightlife |null |null |[] |Nightlife |
pets |null |null |[] |Pets |
professional |null |null |[] |Professional Services |
publicservicesgovt|null |null |[] |Public Services & Government|
religiousorgs |null |null |[] |Religious Organizations |
restaurants |null |null |[] |Restaurants |
shopping |null |null |[] |Shopping |
+------------------+-----------------+-----------------+-------+----------------------------+

### How many top parent categories are there?
In the following cell we use `spark.sql()` to count the number of categories where the size of the `parents` array is 0.

Note that since we are only doing some profiling here - we are not saving the result to a new DataFrame, we did not alias the calculation, so our column name looks like a formula.  If
you are saving the resulting DataFrame to a new variable, you should **ALWAYS** be sure to alias formulas in your `SELECT` clause since otherwise in downstream queries
your column names will look like formulas and that will cause you (and/or your teammates/coworkers) unending grief later on when trying to debug your code or even understand it.

In [0]:
spark.sql("""
SELECT COUNT(alias)
FROM categories
WHERE SIZE(parents) = 0
""").show(30, truncate=False)

+------------+
count(alias)|
+------------+
22 |
+------------+

# Step 5: Categories with multiple parents
Although looking at the first 20 rows did not show any categories with multiple parents, the name of the field in the data
("parents", which is plural) and the fact that the field is an array, should make us suspect that there are categories with
multiple parents.  We want to have a hierarchy, which means each category would have one path to a top-level category.

***We need to figure out how big of an issue this is***

In the following blank cell, add code using the `spark.sql` method to show the details for those 
categories with multiple parents. 

HINT: This is essentially the same as the code for finding the top-level categories, you just need a different 
comparison for the result of the `SIZE(parents)` function.

In [0]:
#Add your query here
spark.sql("""
SELECT *
FROM categories
WHERE SIZE(parents) > 1


""").show(truncate=False)

+---------------+----------------------------------------+-----------------+---------------------------------------+--------------------------+
alias |country_blacklist |country_whitelist|parents |title |
+---------------+----------------------------------------+-----------------+---------------------------------------+--------------------------+
cosmetics |null |null |[shopping, beautysvc] |Cosmetics & Beauty Supply |
deptstores |null |null |[shopping, fashion] |Department Stores |
eltern_cafes |null |[CH, AT, DE] |[restaurants, food] |Parent Cafes |
galleries |null |null |[shopping, arts] |Art Galleries |
hair_extensions|[HK, TR, BE, NL, PL] |null |[hair, beautysvc] |Hair Extensions |
hotels |null |null |[eventservices, hotelstravel] |Hotels |
isps |null |null |[homeservices, professional] |Internet Service Providers|
jazzandblues |null |null |[arts, nightlife] |Jazz & Blues |
kiosk |[HK, IT, FR, BR, GB, US, SG, CA, IE, NZ]|null |[shopping, food] |Kiosk |
medicalspa |null |null |[health, beautysvc] |Medical Spas |
musicvenues |null |null |[arts, nightlife] |Music Venues |
perfume |[TR, SG, PL, CA, FI] |null |[shopping, beautysvc] |Perfume |
sportswear |null |null |[fashion, sportgoods] |Sports Wear |
stationery |null |null |[flowers, eventservices, artsandcrafts]|Cards & Stationery |
swimminglessons|null |null |[fitness, specialtyschools] |Swimming Lessons/Schools |
wineries |[FI] |null |[arts, food] |Wineries |
+---------------+----------------------------------------+-----------------+---------------------------------------+--------------------------+

# Step 6: Filtering the Business Data for Restaurants

If we want to filter the business data to get only businesses that are Restaurants, there is a top level category named `Restaurants`. The problem we have is that there is 
a PySpark and SQL function named `array_contains` that allows us to see if a value is in our array (list), but the business categories field is not an array, 
it's a comma-separated list (scroll up to where you showed this earlier).

#### Splitting a string to generate an array
PySpark has another function we can use to first transform our data; the [`split` function](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.split).  This is a string function that will take a string and split it on some 
other recurring value (in our case, a comma) and return a list of the values.  We have another issue, since the string in business categories has spaces after 
the commas, we need to trim off the spaces.

#### Regular expressions
The `split` function's pattern can be what is known as a regular expression, and these are often used when matching text.  What we want is to split on a comma surrounded by any amount of leading or trailing blank space (so the blanks don't end up in the values we get back).  To do this, the pattern `\s` means a blank space, and we can use the wildcard `*` to mean "zero or more", so the pattern `\s*,\s*` means a comma surrounded by any amount of blank space.

The `\s*` is really important - this means that when we "throw away" the commas, we are also throwing away any blank spaces that would also cause us problems.

In the following cell we create a new DataFrame with the business_id, business' name, and the categories as an array.

In [0]:
df_businessCat = df_business.select("business_id", "name", "state","attributes",f.split(df_business.categories,'\s*,\s*').alias("categories"))
df_businessCat.show(10, truncate=60)

+----------------------+----------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
 business_id| name|state| attributes| categories|
+----------------------+----------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
f9NumwFMBDn751xgFiRbNA| The Range At Lake Norman| NC|[,,,,,,, True,, True, {'garage': False, 'street': False, ...| [Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping]|
Yzvjg0SayhoZgCljUJRF9Q| Carlos Santo, NMD| AZ| [,,,,,,,,,,, True,,,,,,,, True,,,,,,,,,,,,,,,,,,,]|[Health & Medical, Fitness & Instruction, Yoga, Active Li...|
XNoUzKckATkOD1hP6vghZg| Felinus| QC| null| [Pets, Pet Services, Pet Groomers]|
6OAZjbxqM5ol29BuHsil3w| Nevada House of Hose| NV|[,,,,,,, True,, True, {'garage': False, 'street': False, ...|[Hardware Stores, Home Services, Building Supplies, Home ...|
51M2Kk903DFYI6gnB5I6SQ| USE MY GUY SERVICES LLC| AZ| [,,,,,,,,, True,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,]|[Home Services, Plumbing, Electricians, Handyman, Contrac...|
cKyLV5oWZJ2NudWgqs8VZw| Oasis Auto Center - Gilbert| AZ| [,,,,,,,,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,,,]|[Auto Repair, Automotive, Oil Change Stations, Transmissi...|
oiAlXZPIFm2nBCt0DHLu_Q| Green World Cleaners| NV|[,,,,,,, True,, True, {'garage': False, 'street': False, ...| [Dry Cleaning & Laundry, Local Services, Laundry Services]|
ScYkbYNkDgCneBrD9vqhCQ|Junction Tire & Auto Service| AZ| [,,,,,,,,, True,, False,,,,,,,,,,,,,,,,,,,,,,,,,,,]| [Auto Repair, Oil Change Stations, Automotive, Tires]|
pQeaRpvuhoEqudo3uymHIQ| The Empanadas House| IL|[,, 'none',,,,, False,, True, None, False,,,,,,,, False,,...|[Ethnic Food, Food Trucks, Specialty Food, Imported Food,...|
EosRKXIGeSWFYWwpkbhNnA| Xtreme Couture| ON|[,,,,,,, False,,, {'garage': False, 'street': False, 'val...| [Martial Arts, Gyms, Fitness & Instruction, Active Life]|
+----------------------+----------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
only showing top 10 rows

### Step 6b: Splitting in Spark SQL
Most anything you can do in PySpark you can also do in Spark SQL, and splitting is no exception.
We can use the `SPLIT` function to split the string on commans (surrounded by blanks as specified in our regular expression), and then we set a column name alias for the re-structured column using `AS`.

In [0]:
# Add your query here to do the same query using SQL
spark.sql("""
select business_id,name,state, attributes, split(business.categories,'\s*,\s*') as categories
from business

""").show(10, truncate=60)

+----------------------+----------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
 business_id| name|state| attributes| categories|
+----------------------+----------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
f9NumwFMBDn751xgFiRbNA| The Range At Lake Norman| NC|[,,,,,,, True,, True, {'garage': False, 'street': False, ...| [Active Life, Gun/Rifle Range, Guns & Ammo, Shopping]|
Yzvjg0SayhoZgCljUJRF9Q| Carlos Santo, NMD| AZ| [,,,,,,,,,,, True,,,,,,,, True,,,,,,,,,,,,,,,,,,,]|[Health & Medical, Fitness & Instruction, Yoga, Active...|
XNoUzKckATkOD1hP6vghZg| Felinus| QC| null| [Pet, Pet Service, Pet Groomers]|
6OAZjbxqM5ol29BuHsil3w| Nevada House of Hose| NV|[,,,,,,, True,, True, {'garage': False, 'street': False, ...|[Hardware Store, Home Service, Building Supplie, Home ...|
51M2Kk903DFYI6gnB5I6SQ| USE MY GUY SERVICES LLC| AZ| [,,,,,,,,, True,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,]|[Home Service, Plumbing, Electrician, Handyman, Contr...|
cKyLV5oWZJ2NudWgqs8VZw| Oasis Auto Center - Gilbert| AZ| [,,,,,,,,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,,,]|[Auto Repair, Automotive, Oil Change Station, Transmis...|
oiAlXZPIFm2nBCt0DHLu_Q| Green World Cleaners| NV|[,,,,,,, True,, True, {'garage': False, 'street': False, ...| [Dry Cleaning & Laundry, Local Service, Laundry Services]|
ScYkbYNkDgCneBrD9vqhCQ|Junction Tire & Auto Service| AZ| [,,,,,,,,, True,, False,,,,,,,,,,,,,,,,,,,,,,,,,,,]| [Auto Repair, Oil Change Station, Automotive, Tires]|
pQeaRpvuhoEqudo3uymHIQ| The Empanadas House| IL|[,, 'none',,,,, False,, True, None, False,,,,,,,, False,,...|[Ethnic Food, Food Truck, Specialty Food, Imported Foo...|
EosRKXIGeSWFYWwpkbhNnA| Xtreme Couture| ON|[,,,,,,, False,,, {'garage': False, 'street': False, 'val...| [Martial Art, Gym, Fitness & Instruction, Active Life]|
+----------------------+----------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
only showing top 10 rows

### Step 7a: Filtering for Restaurants using PySpark
Now that we the categories column as an array, we can check if it contains `Restaurants`.  PySpark has a function for that!

In PySpark there is an <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.array_contains" target="_blank">`array_contains()` function</a> that takes two parameters:
* the column with the array
* the value we want to check for
This will return a Boolean value True / False that we could use to decide if we want a row.

First we will do this with a PySpark `filter`.

In [0]:
df_businessCat.filter(f.array_contains("categories","Restaurants")).show(truncate=False)

+----------------------+-------------------------------+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
business_id |name |state|attributes |categories |
+----------------------+-------------------------------+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
pQeaRpvuhoEqudo3uymHIQ|The Empanadas House |IL |[,, 'none',,,,, False,, True, None, False,,,,,,,, False,,,, True,, u'quiet',, False, u'casual',, True, False, 1, False,, True,,,] |[Ethnic Food, Food Trucks, Specialty Food, Imported Food, Argentine, Food, Restaurants, Empanadas] |
CsLQLiRoafpJPJSkNX2h5Q|Middle East Deli |NC |[,,,,,,,,, True, {'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False},,,,,,,,, True,,,,,,,, False, u'casual',,, True, 1, False,, True,,,] |[Food, Restaurants, Grocery, Middle Eastern] |
eBEfgOPG7pvFhb2wcG9I7w|Philthy Phillys |ON |[,, u'none',,,,, True,,, {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False},, True,,,, False,,,,,,, True,,,, False,,, True,,, False, False, True,,, u'no'] |[Restaurants, Cheesesteaks, Poutineries] |
lu7vtrp_bE9PnxWfA8g4Pg|Banzai Sushi |ON |[,, u'none',,,,,,,,,,,,,,,,, True,,,,,,,,,,, True, False,, False,, True,,,] |[Japanese, Fast Food, Food Court, Restaurants] |
9sRGfSVEfLhN_km60YruTA|Apadana Restaurant |ON |[,, u'full_bar', {'touristy': False, 'hipster': False, 'romantic': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False},,,,,,, {'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False},, True,,,, False,,,, {'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False},, False, True,,,, True,,, True,,, True, True, True,, True, u'free'] |[Persian/Iranian, Turkish, Middle Eastern, Restaurants, Kebab] |
vjTVxnsQEZ34XjYNS-XUpA|Wetzel's Pretzels |AZ |[,, u'none', {'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': False},,,, True,, True, {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False},,,,,,,,, True,,,, False,,,, False, u'casual',, False, True, 1, False,, True,,,] |[Food, Pretzels, Bakeries, Fast Food, Restaurants] |
fnZrZlqW1Z8iWgTVDfv_MA|Carl's Jr |NV |[,, u'none', {'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True},,,, True,, True, {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False},, False,,,,,,, True,,,, False,, u'average',, False, u'casual',, False, True, 1, False,, True,,, u'no'] |[Mexican, Restaurants, Fast Food] |
rVBPQdeayMYht4Uv_FOLHg|Gourmet Burger Company |ON |[,, u'none', {'roman

### Step 7b: Filtering for Restaurants using Spark SQL
Run the following cell to create a temporary view named `dbusinessCat`.

In [0]:
df_businessCat.createOrReplaceTempView("businessCat")

In the following cell, use spark.sql to run a query that creates a new DataFrame named df_restaurants:
* Selects all of the records in `businessCat`
* Has a WHERE clause that uses `ARRAY_CONTAINS()` to check if the `categories` column to see if it contains the string "Restaurants".

In [0]:
# Add your query here to get the restaurants

df_restaurants = spark.sql("""
select * from businessCat 
where array_contains(categories,'Restaurants') 

""")
df_restaurants.show(truncate=60)

+----------------------+-------------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
 business_id| name|state| attributes| categories|
+----------------------+-------------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
pQeaRpvuhoEqudo3uymHIQ| The Empanadas House| IL|[,, 'none',,,,, False,, True, None, False,,,,,,,, False,,...|[Ethnic Food, Food Trucks, Specialty Food, Imported Food,...|
CsLQLiRoafpJPJSkNX2h5Q| Middle East Deli| NC|[,,,,,,,,, True, {'garage': False, 'street': False, 'vali...| [Food, Restaurants, Grocery, Middle Eastern]|
eBEfgOPG7pvFhb2wcG9I7w| Philthy Phillys| ON|[,, u'none',,,,, True,,, {'garage': False, 'street': Fals...| [Restaurants, Cheesesteaks, Poutineries]|
lu7vtrp_bE9PnxWfA8g4Pg| Banzai Sushi| ON|[,, u'none',,,,,,,,,,,,,,,,, True,,,,,,,,,,, True, False,...| [Japanese, Fast Food, Food Court, Restaurants]|
9sRGfSVEfLhN_km60YruTA| Apadana Restaurant| ON|[,, u'full_bar', {'touristy': False, 'hipster': False, 'r...|[Persian/Iranian, Turkish, Middle Eastern, Restaurants, K...|
vjTVxnsQEZ34XjYNS-XUpA| Wetzel's Pretzels| AZ|[,, u'none', {'romantic': False, 'intimate': False, 'tour...| [Food, Pretzels, Bakeries, Fast Food, Restaurants]|
fnZrZlqW1Z8iWgTVDfv_MA| Carl's Jr| NV|[,, u'none', {'romantic': False, 'intimate': False, 'clas...| [Mexican, Restaurants, Fast Food]|
rVBPQdeayMYht4Uv_FOLHg| Gourmet Burger Company| ON|[,, u'none', {'romantic': False, 'intimate': False, 'clas...| [Restaurants, Burgers, Food]|
98hyK2QEUeI8v2y0AghfZA|Pho Lee's Vietnamese Restaurant| OH|[,, u'none', {'touristy': False, 'hipster': False, 'roman...| [Restaurants, Vietnamese, Soup]|
fhNf_sg-XzZ3e7HEVGuOZg| Meat Chix And Wieners| NV|[,, u'none', {'romantic': False, 'intimate': False, 'tour...|[Burgers, Restaurants, American (Traditional), Chicken Wi...|
LoRef3ChgZKbxUio-sHgQg| Amir| QC|[,,, {'romantic': False, 'intimate': False, 'classy': Fal...| [Restaurants, Lebanese, Middle Eastern]|
Ga2Bt7xfqoggTypWD5VpoQ| Amando's Bros| AZ|[,, u'none', {'romantic': False, 'intimate': False, 'clas...| [Mexican, Restaurants]|
xFc50drSPxXkcLvX5ygqrg| Boomer's Sweet Home Chicago| AZ|[,, u'none',,,,,,, True, {'garage': False, 'street': Fals...| [Burgers, Fast Food, American (New), Restaurants, Hot Dogs]|
tLpkSwdtqqoXwU0JAGnApw| Wendy's| OH|[,, u'none', {'romantic': False, 'intimate': False, 'tour...| [Restaurants, Fast Food, Burgers]|
Sd75ucXKoZUM2BEfBHFUOg| China Gourmet| AZ|[,,,,,,, False,, True, {'garage': False, 'street': False,...| [Chinese, Restaurants]|
lK-wuiq8b1TuU7bfbQZgsg| Hingetown| OH|[,, u'none', {'romantic': False, 'intimate': False, 'clas...|[Shopping Centers, Food, Coffee & Tea, Cafes, Museums, Re...|
LAoSegVNU4wx4GTA8reB6A| Tzikii Food Truck| AZ|[,,,,,,,,, True,, False,,,,,,,,, {'dessert': False, 'late...| [Restaurants, Egyptian, Food Trucks, Food, Middle Eastern]|
-qjn24n8HYF6It9GQrQntw| Five Guys Burgers and Fries| NC|[,, u'none', {'romantic': False, 'intimate': False, 'tour...| [Fast Food, American (New), Restaurants, Burgers]|
ZkzutF0P_u0C0yTulwaHkA| Lelulos Pizzeria| PA|[,, 'none', {'romantic': False, 'intimate': False, 'class...| [Restaurants, Pizza]|
0QjROMVW9ACKjhSEfHqNCQ| Mi Mi Restaurant| ON|[,, u'beer_and_wine', {'romantic': False, 'intimate': Fal...| [Vietnamese, Restaurants]|
+----------------------+-------------------------------+-----+------------------------------------------------------------+------------------------------------------------------------+
only showing top 20 rows

# Step 8: Extracting the RestaurantsTakeOut Attribute

From printing the schema earlier, we can see that one of the attributes for businesses is `RestaurantsTakeOut`  As we did in Trifacta Wrangler, we 
can extract that attribute as a separate column.  What was easier in Wrangler was to get a count of how often each attribute occurred.  That could be done in Spark
directly, but it would take a bit more work.

As in Python, to get the `RestaurantsTakeOut` attribute we use the dot notation oto get the attribute.

In [0]:
df_restaurants.createOrReplaceTempView("restaurants")

In [0]:
# Add your query here
df_restaurants_takeout = spark.sql("""
select business_id, name,state, categories,attributes.RestaurantsTakeOut as Takeout
from restaurants
""")
df_restaurants_takeout.show(truncate=80)

+----------------------+-------------------------------+-----+--------------------------------------------------------------------------------+-------+
 business_id| name|state| categories|Takeout|
+----------------------+-------------------------------+-----+--------------------------------------------------------------------------------+-------+
pQeaRpvuhoEqudo3uymHIQ| The Empanadas House| IL|[Ethnic Food, Food Trucks, Specialty Food, Imported Food, Argentine, Food, Re...| True|
CsLQLiRoafpJPJSkNX2h5Q| Middle East Deli| NC| [Food, Restaurants, Grocery, Middle Eastern]| True|
eBEfgOPG7pvFhb2wcG9I7w| Philthy Phillys| ON| [Restaurants, Cheesesteaks, Poutineries]| True|
lu7vtrp_bE9PnxWfA8g4Pg| Banzai Sushi| ON| [Japanese, Fast Food, Food Court, Restaurants]| True|
9sRGfSVEfLhN_km60YruTA| Apadana Restaurant| ON| [Persian/Iranian, Turkish, Middle Eastern, Restaurants, Kebab]| True|
vjTVxnsQEZ34XjYNS-XUpA| Wetzel's Pretzels| AZ| [Food, Pretzels, Bakeries, Fast Food, Restaurants]| True|
fnZrZlqW1Z8iWgTVDfv_MA| Carl's Jr| NV| [Mexican, Restaurants, Fast Food]| True|
rVBPQdeayMYht4Uv_FOLHg| Gourmet Burger Company| ON| [Restaurants, Burgers, Food]| True|
98hyK2QEUeI8v2y0AghfZA|Pho Lee's Vietnamese Restaurant| OH| [Restaurants, Vietnamese, Soup]| True|
fhNf_sg-XzZ3e7HEVGuOZg| Meat Chix And Wieners| NV| [Burgers, Restaurants, American (Traditional), Chicken Wings]| True|
LoRef3ChgZKbxUio-sHgQg| Amir| QC| [Restaurants, Lebanese, Middle Eastern]| True|
Ga2Bt7xfqoggTypWD5VpoQ| Amando's Bros| AZ| [Mexican, Restaurants]| True|
xFc50drSPxXkcLvX5ygqrg| Boomer's Sweet Home Chicago| AZ| [Burgers, Fast Food, American (New), Restaurants, Hot Dogs]| True|
tLpkSwdtqqoXwU0JAGnApw| Wendy's| OH| [Restaurants, Fast Food, Burgers]| True|
Sd75ucXKoZUM2BEfBHFUOg| China Gourmet| AZ| [Chinese, Restaurants]| True|
lK-wuiq8b1TuU7bfbQZgsg| Hingetown| OH|[Shopping Centers, Food, Coffee & Tea, Cafes, Museums, Restaurants, Shopping,...| False|
LAoSegVNU4wx4GTA8reB6A| Tzikii Food Truck| AZ| [Restaurants, Egyptian, Food Trucks, Food, Middle Eastern]| False|
-qjn24n8HYF6It9GQrQntw| Five Guys Burgers and Fries| NC| [Fast Food, American (New), Restaurants, Burgers]| True|
ZkzutF0P_u0C0yTulwaHkA| Lelulos Pizzeria| PA| [Restaurants, Pizza]| True|
0QjROMVW9ACKjhSEfHqNCQ| Mi Mi Restaurant| ON| [Vietnamese, Restaurants]| True|
+----------------------+-------------------------------+-----+--------------------------------------------------------------------------------+-------+
only showing top 20 rows

### Step 8b: What are the Attribute values for RestaurantsTakeOut?
Now that we extracted the take out attribute,it would be good to know what the possible values are in that column.
* Select just the takeOut column
* Get the distinct values
* Show the results
* How many for each value?
* How could you clean this up?

In [0]:
df_restaurants_takeout.createOrReplaceTempView("take_out")

In [0]:
# Add your query here
spark.sql("""
Select Takeout, count(*) from take_out
group by TakeOut


""").show()

+-------+--------+
Takeout|count(1)|
+-------+--------+
 None| 162|
 False| 3478|
 null| 8422|
 True| 51882|
+-------+--------+

### Step 9: Counting Your Favorite Type of Restaurant (what cuisine they serve) By State
* Find your favorite cuisine in the categories list: <a href="https://www.yelp.com/developers/documentation/v3/all_category_list" target="_blank">https://www.yelp.com/developers/documentation/v3/all_category_list</a>
* clean up the takeOut values <br/>**HINT:** There is a SQL function named IF that has the same format as in Excel:<br/>IF(some condition **,** then value **,** else value )
* Create a query to get the businesses that are restaurants serving your favorite cuisine
* Count the restaurants grouped by state and whether they serve take out
* Show the states in order of the state and take out status

In [0]:
#Add your query here to clean the values in the takeOut column
df_takeout_cleaned = spark.sql("""
select business_id,state , name, categories, takeout  from take_out
where array_contains(categories,'Fast Food')


""")
df_takeout_cleaned.show(truncate=False)

+----------------------+-----+---------------------------+-------------------------------------------------------------------------+-------+
business_id |state|name |categories |takeout|
+----------------------+-----+---------------------------+-------------------------------------------------------------------------+-------+
lu7vtrp_bE9PnxWfA8g4Pg|ON |Banzai Sushi |[Japanese, Fast Food, Food Court, Restaurants] |True |
vjTVxnsQEZ34XjYNS-XUpA|AZ |Wetzel's Pretzels |[Food, Pretzels, Bakeries, Fast Food, Restaurants] |True |
fnZrZlqW1Z8iWgTVDfv_MA|NV |Carl's Jr |[Mexican, Restaurants, Fast Food] |True |
xFc50drSPxXkcLvX5ygqrg|AZ |Boomer's Sweet Home Chicago|[Burgers, Fast Food, American (New), Restaurants, Hot Dogs] |True |
tLpkSwdtqqoXwU0JAGnApw|OH |Wendy's |[Restaurants, Fast Food, Burgers] |True |
-qjn24n8HYF6It9GQrQntw|NC |Five Guys Burgers and Fries|[Fast Food, American (New), Restaurants, Burgers] |True |
MTx-Zdl_KcU_z9G832XAjg|NV |Taco Bell |[Fast Food, Restaurants] |True |
0DnQh8SE8BSnvJltGCCiWg|ON |Chick-N-Joy |[Fast Food, Restaurants, Chicken Shop] |True |
5X7TQsA3iDbBc8yy2YkCnA|AZ |Blimpie |[Restaurants, Sandwiches, Fast Food] |True |
0nqQIM2PvOcTX6v0L537_Q|ON |Fearless Meat |[Fast Food, Burgers, Sandwiches, Restaurants] |True |
TKKX9bllRBMzp0NYr441PQ|ON |Mi'Hito Sushi Laboratory |[Food, Poke, Fast Food, Sushi Bars, Restaurants, Japanese, Asian Fusion] |True |
ypILNgy7QFskKAdcPKB2RQ|SC |KFC |[Restaurants, Fast Food, Chicken Shop, Chicken Wings] |True |
2nbibYLTR96Nr2nk-1spdw|AZ |Church's Chicken |[Restaurants, Fast Food, Chicken Wings, Chicken Shop, Breakfast & Brunch]|True |
W-5cdzVAeRv0EtAeed4tqw|AZ |Sam's Gyros |[Hot Dogs, Fast Food, Restaurants, Middle Eastern] |True |
CfwrsG76Wm4iLS22v_wAcg|OH |McDonald's |[Restaurants, Coffee & Tea, Burgers, Fast Food, Food] |True |
_V4CzzA7Z9h4qyLIdG-KUg|NC |Wendy's |[Restaurants, Hot Dogs, Burgers, Fast Food] |True |
grZ6FnfZoj1pQWElAQve3g|PA |Wendy's |[Fast Food, Restaurants, Burgers] |True |
K4rLi2Mqz931v07nMSih5w|AZ |Chipotle Mexican Grill |[Fast Food, Restaurants, Mexican] |True |
CU4LgHxH0EwXnPUmQsWFSQ|AZ |Church's Chicken |[Fast Food, Chicken Shop, Restaurants, Chicken Wings, Breakfast & Brunch]|null |
Rtaca3VZIioxy6N006NKaQ|AZ |Sonic Drive-In |[Food, Burgers, Ice Cream & Frozen Yogurt, Restaurants, Fast Food] |True |
+----------------------+-----+---------------------------+-------------------------------------------------------------------------+-------+
only showing top 20 rows

In [0]:
df_takeout_cleaned.createOrReplaceTempView("take_out_cleaned")

In [0]:
#Add your query here to summarize the number of restaurants by state that have take out for your favorite cuisine
spark.sql("""
select state , if(takeout is null or takeout='none', 'False',takeout) as takeOut,  count(*) as Fast_food
from take_out_cleaned
where array_contains(categories,'Fast Food')
group by state,takeout
order by state, takeout




""").show(truncate=False)

+-----+-------+---------+
state|takeOut|Fast_food|
+-----+-------+---------+
AB |False |62 |
AB |False |17 |
AB |True |266 |
AL |False |1 |
AZ |False |106 |
AZ |False |17 |
AZ |None |4 |
AZ |True |1944 |
BC |True |1 |
CA |True |1 |
IL |False |17 |
IL |False |3 |
IL |True |92 |
NC |False |93 |
NC |False |3 |
NC |True |698 |
NV |False |56 |
NV |False |14 |
NV |None |1 |
NV |True |1272 |
+-----+-------+---------+
only showing top 20 rows